__TODO__
- [ ] Include side (left/right) were participant seat on final participants list (load from participants_full.csv)

__Dependencies__

In [1]:
import copy  # copy big/deep objects by value
import datetime  # datetime operations
import os  # OS operations (read/write files/folders)

import pandas as pd  # operate with dataframes
from IPython.display import display  # print nicely
from tqdm.notebook import tqdm  # mother of progressbars for Python

__Colliders (ray-traced VR participants data) cleanup__

Get colliders filenames, condition, N frames

In [2]:
path_c = "./data/colliders"  # ray-traced data path
# get all ray-traced data from the folder "data/colliders"
cs = os.listdir(path_c)
for f in cs:
    # filter hidden/config files and folders
    if f.startswith(".") or not f.endswith(".csv"):
        cs.remove(f)  # remove hidden/config file

# progress bar format definitons
m_format = """📄 {n_fmt} of {total_fmt} {desc} processed: {bar}
            {percentage:3.0f}% ⏱️{elapsed} ⏳{remaining} 📅{eta:%d/%m/%y}
             🕒{eta:%H:%M}"""

# cs progress bar
cs_progress = tqdm(
    cs,
    desc="📂 files",
    dynamic_ncols=False,
    ncols="100%",
    mininterval=0.001,
    bar_format=m_format,
)

# create df to store vr participants information
colliders_cols = ["file", "created", "condition", "frames"]
wd_colls = pd.DataFrame(columns=colliders_cols)
wd_colls.index.name = "uid"

# iterate over all files to remove hidden/config files and folders from the lists
for f in cs_progress:  # iterate over cs_progress as it's build from cs
    current = f.split(".csv")[0]  # remove file extension
    current = current.split("-")  # split filename by "-"
    # get uid, condition filename, and creation date from file
    uid = current[-1]
    condition = current[-2]
    f_path = f"{path_c}/{f}"
    created = os.path.getmtime(f_path)  # creation timestamp
    created = datetime.datetime.fromtimestamp(created)  # translate as datetime
    created = created.strftime("%d.%m.%Y %H:%M")  # arrange it
    c_data = pd.read_csv(f_path)  # CSV to dataframe
    n_frames = c_data["frameNumber"].iloc[-1]  # last stored frame number
    # add new VR participant information into the df
    wd_colls.loc[uid] = {
        "file": f,
        "created": created,
        "condition": condition,
        "frames": n_frames,
    }

# create vr participants counter df
n_colls = pd.DataFrame(columns=["avas", "radio", "taxi", "total"])
n_colls.index.name = "colliders"

# get counters
avas_c = wd_colls[wd_colls.condition == "AVAS"].index.size
radio_c = wd_colls[wd_colls.condition == "RadioTalk"].index.size
taxi_c = wd_colls[wd_colls.condition == "TaxiDriver"].index.size
wd_colls_c = wd_colls.index.size

# add and show them
counters = [avas_c, radio_c, taxi_c, wd_colls_c]
n_colls.loc["raw"] = counters
display(n_colls)

wd_colls  # preview df

,avas,radio,taxi,total
colliders,,,,
raw,8896,8796,4428,22120


,file,created,condition,frames
uid,,,,
ce2aae5aa75048d3bdc419cb985f785a,Path_ MS Wissenschaft-RadioTalk-ce2aae5aa75048...,10.03.2020 00:38,RadioTalk,3070
b7377d501fae4b03965a87432b5b587f,Path_ MS Wissenschaft-RadioTalk-b7377d501fae4b...,18.02.2020 19:23,RadioTalk,2317
4496e5d070e84762880f864b46336eb5,Path_ MS Wissenschaft-RadioTalk-4496e5d070e847...,20.02.2020 18:42,RadioTalk,3070
b1fce79dd2b84d11b91c76a583514032,Path_ MS Wissenschaft-RadioTalk-b1fce79dd2b84d...,10.03.2020 00:08,RadioTalk,3070
2e85ba9bdb694897bd1083b16160d312,Path_ MS Wissenschaft-TaxiDriver-2e85ba9bdb694...,16.04.2020 03:10,TaxiDriver,3070
...,...,...,...,...
75c9b428857246e4a37ce3a2116892c8,Path_ MS Wissenschaft-AVAS-75c9b428857246e4a37...,18.02.2020 12:53,AVAS,3070
4c8749ec4fce4655ab1932691ef24c2e,Path_ MS Wissenschaft-RadioTalk-4c8749ec4fce46...,25.02.2020 17:45,RadioTalk,3070
2bb97472cd9048f3950b56f4c8a9d0da,Path_ MS Wissenschaft-TaxiDriver-2bb97472cd904...,20.02.2020 23:24,TaxiDriver,3070


Keep only finished VR experiments (N frames = 3070)

In [3]:
wd_colls = wd_colls[wd_colls.frames == 3070]

avas_c = wd_colls[wd_colls.condition == "AVAS"].index.size
radio_c = wd_colls[wd_colls.condition == "RadioTalk"].index.size
taxi_c = wd_colls[wd_colls.condition == "TaxiDriver"].index.size
wd_colls_c = wd_colls.index.size

counters = [avas_c, radio_c, taxi_c, wd_colls_c]
n_colls.loc["finished"] = counters

n_colls

,avas,radio,taxi,total
colliders,,,,
raw,8896,8796,4428,22120
finished,8288,7928,4142,20358


Look for duplicated IDs

In [4]:
# get duplicated IDs and preview them
duplicated = wd_colls[wd_colls.index.duplicated(keep=False)]

if duplicated.index.size > 0:
    display(duplicated)
else:
    print("There are no duplicated IDs.")

There are no duplicated IDs.


__Questionnaires cleanup__

Read data, show total stored participants (raw/uncleaned).

In [5]:
# raw (uncleaned) questionnaire paths
msw_q_path = "./data/questionnaires/msw-answers.csv"
bmbf_q_path = "./data/questionnaires/bmbf-answers.csv"

# load questionnaire data
msw_q = pd.read_csv(msw_q_path)
bmbf_q = pd.read_csv(bmbf_q_path)

# create dataframe to add participant counters during data cleanup
participants_cols = ["msw", "bmbf", "total"]
n_quests = pd.DataFrame(columns=participants_cols)
n_quests.index.name = "questionnaries"

# dataframes row counts
msw_c = msw_q.index.size
bmbf_c = bmbf_q.index.size
counters = [msw_c, bmbf_c, msw_c + bmbf_c]  # arrange as list with total
n_quests.loc["raw"] = counters  # add current counters

# show number of participants table
display(n_quests)

# show first 5 rows of each df
display(msw_q.head())
display(bmbf_q.head())

,msw,bmbf,total
questionnaries,,,
raw,8873,1209,10082


,id,uid,date,Intention to Use 1,Perceived Usefulness 4,Perceived Ease of Use 2,Trust,Sex,Age,Aviophobia,Driving Frequency,Playing Hours,VR Playing Frequency
0,1,1234567890qwertyuiop,2019-04-23 13:25:43,100,0,50,NaN,Keine Angabe,50,0,0,60,mehr als 10 mal
1,2,1234567890qwertyuiop,2019-04-23 14:13:16,100,52,53,NaN,Intersex,43,1,47,60,unter 10
2,3,3b90576c-0c96-46ba-b07a-9b3c8fc6a5c0,2019-04-23 18:43:07,100,51,49,NaN,Intersex,43,0,0,60,unter 10
3,4,3b90576c-0c96-46ba-b07a-9b3c8fc6a5c5,2019-04-24 18:35:22,74,100,91,NaN,Intersex,34,0,17,45,mehr als 10 mal
4,5,3b90576c-0c96-46ba-b07a-9b3c8fc6a5c0,2019-04-24 18:45:33,100,18,19,NaN,Keine Angabe,0,1,0,0,Ein mal


,id,uid,date,Intention to Use 1,Perceived Usefulness 4,Perceived Ease of Use 2,Trust,Sex,Age,Aviophobia,Driving Frequency,Playing Hours,VR Playing Frequency
0,1,99,2019-07-02 10:24:12,75,50,43,100,intersex,12,1,3,2,Ein mal
1,2,99,2019-07-02 10:27:51,50,100,0,97,weiblich,86,1,6,53,Ein mal
2,3,99,2019-07-02 10:56:13,83,100,82,7,weiblich,1,1,2,2,Ein mal
3,4,99,2019-07-02 10:58:10,77,33,43,16,weiblich,1,1,2,1,Ein mal
4,5,99,2019-07-02 11:00:09,79,78,90,61,weiblich,1,1,3,3,unter 10


Remove id column, set uid as index, remove all data before starting the exhibitions.

In [6]:
# remove column id (autoincrement)
msw_q.drop(columns=["id"], inplace=True)
bmbf_q.drop(columns=["id"], inplace=True)

# set uid as index
msw_q.set_index("uid", inplace=True)
bmbf_q.set_index("uid", inplace=True)

# select only data from the start of each expo
msw_q = msw_q.loc[msw_q.date > "2019-05-14"]
bmbf_q = bmbf_q.loc[bmbf_q.date > "2019-07-10"]

# dataframes row counts
msw_c = msw_q.index.size
bmbf_c = bmbf_q.index.size

counters = [msw_c, bmbf_c, msw_c + bmbf_c]  # arrange as list with total
n_quests.loc["expo"] = counters  # add current counters

# show number of participants table
display(n_quests)

# show first 5 rows of each df
display(msw_q.head())
display(bmbf_q.head())

,msw,bmbf,total
questionnaries,,,
raw,8873,1209,10082
expo,8835,1153,9988


,date,Intention to Use 1,Perceived Usefulness 4,Perceived Ease of Use 2,Trust,Sex,Age,Aviophobia,Driving Frequency,Playing Hours,VR Playing Frequency
uid,,,,,,,,,,,
8598a83e2af441b8bcd0ae5d84beb875,2019-05-14 15:54:53,100,0,100,NaN,männlich,30,0,5,10,Ein mal
42bac596059749b5b8e8e83ae61de9b4,2019-05-14 15:58:49,50,34,83,NaN,weiblich,47,0,30,0,Ein mal
586c107173344c59aa4f71e3573233f0,2019-05-14 15:59:00,2,3,1,NaN,weiblich,37,0,0,0,mehr als 10 mal
9cdd85098b0b4ad5ab2282a5ac371a5e,2019-05-14 16:02:09,19,21,50,NaN,weiblich,40,0,22,0,Noch nie
ff846d92c7e6471183595bd2678f29f6,2019-05-14 16:40:20,0,0,50,NaN,weiblich,40,0,22,2,mehr als 10 mal


,date,Intention to Use 1,Perceived Usefulness 4,Perceived Ease of Use 2,Trust,Sex,Age,Aviophobia,Driving Frequency,Playing Hours,VR Playing Frequency
uid,,,,,,,,,,,
1890c7b8c5004f30bd551c8c83ff7392,2019-07-10 01:48:34,50,25,1,50,männlich,60,1,38,0,Noch nie
b2e4d0f90a74498e9f978fafb36e9960,2019-07-10 17:19:47,23,50,50,78,weiblich,23,1,6,0,unter 10
a3a133cbc9d4496eb0f1a366461f3bd5,2019-07-10 17:24:13,23,34,50,36,weiblich,54,0,30,0,unter 10
cf8bea70b941498a9a6859dff1b2d445,2019-07-10 19:07:51,100,84,100,50,männlich,58,1,40,0,unter 10
1ab45d75eb2b4504a98fc6d8173e5130,2019-07-10 19:33:18,100,86,70,100,männlich,25,0,0,9,unter 10


Add `MSW/BMBF` identifier column, merge dataframes, rename columns, translate values

In [7]:
# short column names mapping
columns_short = {
    "Intention to Use 1": "Intention",
    "Perceived Usefulness 4": "Usefulness",
    "Perceived Ease of Use 2": "Ease",
    "Trust": "Trust",
    "Driving Frequency": "Driving",
    "Playing Hours": "Play",
    "VR Playing Frequency": "VR",
    "condition": "Condition",
    "Sex": "Gender",
}

# gender and vr values translation (DE->EN)
gender_translation = {
    "männlich": "Male",
    "weiblich": "Female",
    "intersex": "intersex",
    "keine Angabe": "N/A",
}
vr_translation = {
    "Noch nie": "never",
    "Ein mal": "once",
    "unter 10": "less than 10 times",
    "mehr als 10 mal": "more than 10 times",
}

# add expo identifier column
msw_q["expo"] = ["MSW" for row in range(msw_q.index.size)]
bmbf_q["expo"] = ["BMBF" for row in range(bmbf_q.index.size)]

# merge data
wd_quest = msw_q.append(bmbf_q)

# rename (shorten) column namnes
wd_quest.rename(columns=columns_short, inplace=True)

# translate column values
wd_quest.Gender = wd_quest.Gender.map(gender_translation)
wd_quest.VR = wd_quest.VR.map(vr_translation)

# dataframe row counts (each expo + total)
msw_c = wd_quest[wd_quest.expo == "MSW"].index.size
bmbf_c = wd_quest[wd_quest.expo == "BMBF"].index.size
wd_quest_c = wd_quest.index.size

counters = [msw_c, bmbf_c, wd_quest_c]  # arrange as list with total
n_quests.loc["merged"] = counters  # add current counters

# show number of participants table
display(n_quests)

wd_quest  # arranged dataframe preview

,msw,bmbf,total
questionnaries,,,
raw,8873,1209,10082
expo,8835,1153,9988
merged,8835,1153,9988


,date,Intention,Usefulness,Ease,Trust,Gender,Age,Aviophobia,Driving,Play,VR,expo
uid,,,,,,,,,,,,
8598a83e2af441b8bcd0ae5d84beb875,2019-05-14 15:54:53,100,0,100,NaN,Male,30,0,5,10,once,MSW
42bac596059749b5b8e8e83ae61de9b4,2019-05-14 15:58:49,50,34,83,NaN,Female,47,0,30,0,once,MSW
586c107173344c59aa4f71e3573233f0,2019-05-14 15:59:00,2,3,1,NaN,Female,37,0,0,0,more than 10 times,MSW
9cdd85098b0b4ad5ab2282a5ac371a5e,2019-05-14 16:02:09,19,21,50,NaN,Female,40,0,22,0,never,MSW
ff846d92c7e6471183595bd2678f29f6,2019-05-14 16:40:20,0,0,50,NaN,Female,40,0,22,2,more than 10 times,MSW
...,...,...,...,...,...,...,...,...,...,...,...,...
eda9175e9e1d4153ac2ca3038b2ab008,2019-12-23 09:23:08,65,30,33,20.0,Male,12,0,0,25,once,BMBF
b8c9e625f07444cc8d4a0fc245bab5dd,2019-12-23 09:28:25,100,50,50,100.0,Female,12,1,99,9,less than 10 times,BMBF
2cd78ad9a37945de9f8cfd5f81c42082,2019-12-27 12:56:23,86,20,71,69.0,Male,57,0,39,0,never,BMBF


Drop participants with duplicated ids

In [8]:
# get duplicated IDs and preview them
duplicated = wd_quest[wd_quest.index.duplicated(keep=False)]  # N = 104
display(duplicated)

# drop them and preview the rest
wd_quest = wd_quest.drop(duplicated.index)
display(wd_quest)

# dataframe row counts (each expo + total)
msw_c = wd_quest[wd_quest.expo == "MSW"].index.size
bmbf_c = wd_quest[wd_quest.expo == "BMBF"].index.size
wd_quest_c = wd_quest.index.size

counters = [msw_c, bmbf_c, wd_quest_c]  # arrange as list with total
n_quests.loc["no duplicates"] = counters  # add current counters

# show number of participants table
n_quests

,date,Intention,Usefulness,Ease,Trust,Gender,Age,Aviophobia,Driving,Play,VR,expo
uid,,,,,,,,,,,,
18881d3ec46e4a34bf3d1ff80de5109a,2019-05-16 09:13:04,100,100,100,NaN,Male,50,0,4,0,more than 10 times,MSW
18881d3ec46e4a34bf3d1ff80de5109a,2019-05-16 09:14:03,100,100,80,NaN,Male,10,0,0,1,more than 10 times,MSW
4a81013077fc458f8b22ea3c9cc686fa,2019-05-16 10:34:02,50,50,50,NaN,intersex,0,1,0,99,less than 10 times,MSW
4a81013077fc458f8b22ea3c9cc686fa,2019-05-16 10:34:13,94,50,50,NaN,Male,8,1,0,14,never,MSW
1ba35aa9ceac4c6a85f55dcee0e8ded2,2019-05-18 18:09:23,0,0,0,NaN,NaN,0,0,0,0,NaN,MSW
...,...,...,...,...,...,...,...,...,...,...,...,...
847d87275dd1461e8aa87f41cc4a70a8,2019-10-18 15:34:09,50,80,60,70.0,Male,14,0,0,20,less than 10 times,MSW
4885a09a14d2454c8f06bee7ee0a5b47,2019-10-21 10:20:29,100,100,100,100.0,Male,11,0,8,0,never,MSW
4885a09a14d2454c8f06bee7ee0a5b47,2019-10-21 10:21:22,0,50,29,100.0,Female,12,1,3,1,less than 10 times,MSW


,date,Intention,Usefulness,Ease,Trust,Gender,Age,Aviophobia,Driving,Play,VR,expo
uid,,,,,,,,,,,,
8598a83e2af441b8bcd0ae5d84beb875,2019-05-14 15:54:53,100,0,100,NaN,Male,30,0,5,10,once,MSW
42bac596059749b5b8e8e83ae61de9b4,2019-05-14 15:58:49,50,34,83,NaN,Female,47,0,30,0,once,MSW
586c107173344c59aa4f71e3573233f0,2019-05-14 15:59:00,2,3,1,NaN,Female,37,0,0,0,more than 10 times,MSW
9cdd85098b0b4ad5ab2282a5ac371a5e,2019-05-14 16:02:09,19,21,50,NaN,Female,40,0,22,0,never,MSW
ff846d92c7e6471183595bd2678f29f6,2019-05-14 16:40:20,0,0,50,NaN,Female,40,0,22,2,more than 10 times,MSW
...,...,...,...,...,...,...,...,...,...,...,...,...
eda9175e9e1d4153ac2ca3038b2ab008,2019-12-23 09:23:08,65,30,33,20.0,Male,12,0,0,25,once,BMBF
b8c9e625f07444cc8d4a0fc245bab5dd,2019-12-23 09:28:25,100,50,50,100.0,Female,12,1,99,9,less than 10 times,BMBF
2cd78ad9a37945de9f8cfd5f81c42082,2019-12-27 12:56:23,86,20,71,69.0,Male,57,0,39,0,never,BMBF


,msw,bmbf,total
questionnaries,,,
raw,8873,1209,10082
expo,8835,1153,9988
merged,8835,1153,9988
no duplicates,8731,1153,9884


Drop NaNs

In [9]:
wd_quest = wd_quest.dropna()  # drop NaNs (inplace=True throws warning)

# set Trust column type to int (read as float)
wd_quest.Trust = wd_quest.Trust.astype("int64")

# dataframe row counts (each expo + total)
msw_c = wd_quest[wd_quest.expo == "MSW"].index.size
bmbf_c = wd_quest[wd_quest.expo == "BMBF"].index.size
wd_quest_c = wd_quest.index.size

counters = [msw_c, bmbf_c, wd_quest_c]  # arrange as list with total
n_quests.loc["no NaNs"] = counters  # add current counters

# show number of participants table
display(n_quests)

wd_quest  # show current df

,msw,bmbf,total
questionnaries,,,
raw,8873,1209,10082
expo,8835,1153,9988
merged,8835,1153,9988
no duplicates,8731,1153,9884
no NaNs,8364,1153,9517


,date,Intention,Usefulness,Ease,Trust,Gender,Age,Aviophobia,Driving,Play,VR,expo
uid,,,,,,,,,,,,
3b90576c-0c96-46ba-b07a-9b3c8fc6a9c9,2019-05-24 17:44:45,50,50,50,50,intersex,90,0,36,33,less than 10 times,MSW
ec61bab6e46741969bceb80ee2246983,2019-05-26 12:06:22,50,50,50,50,intersex,25,1,80,58,more than 10 times,MSW
1fbde03890d94bddaf7aa987fc732607,2019-05-26 16:02:45,100,100,100,96,Male,13,0,0,5,once,MSW
1ad9eea672a94518b150925bac41fcbc,2019-05-26 17:23:38,17,22,0,86,Female,24,0,4,2,once,MSW
461b2d80132f47958e50003c16084cf4,2019-05-26 17:27:34,30,70,80,80,Male,22,0,3,1,less than 10 times,MSW
...,...,...,...,...,...,...,...,...,...,...,...,...
eda9175e9e1d4153ac2ca3038b2ab008,2019-12-23 09:23:08,65,30,33,20,Male,12,0,0,25,once,BMBF
b8c9e625f07444cc8d4a0fc245bab5dd,2019-12-23 09:28:25,100,50,50,100,Female,12,1,99,9,less than 10 times,BMBF
2cd78ad9a37945de9f8cfd5f81c42082,2019-12-27 12:56:23,86,20,71,69,Male,57,0,39,0,never,BMBF


Display all unique inputted values to check if wrong values

In [10]:
filter_out = ["date", "expo"]  # dates and exhibition columns to filter out
for column in wd_quest.columns.to_list():  # for each column of the df
    if column not in filter_out:  # apply filter
        # display column name and inputted unique values
        print(f"{column}: \n{sorted(wd_quest[column].unique())}\n")

Intention: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

Usefulness: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]

Ease: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,

__Merge datasets__

Check and match questionnaires and colliders datasets by id

In [11]:
wd_quest_ids = wd_quest.index  # questionnaire ids

# select only colliders ids that match with the questionnaires
# wd_colls.index.intersection(wd_quest_ids) passed as index because
# .loc doesn't work anymore if there are missing (not matching) indices
wd_colls = wd_colls.loc[wd_colls.index.intersection(wd_quest_ids)]

# merge both dataframes
wd_data = wd_quest.merge(wd_colls, left_on="uid", right_on="uid", how="left")

# drop new NaNs (unmatched ids)
wd_data = wd_data.dropna()

# update wd_quest dataframe
wd_quest = copy.deepcopy(wd_data)

wd_data

,date,Intention,Usefulness,Ease,Trust,Gender,Age,Aviophobia,Driving,Play,VR,expo,file,created,condition,frames
uid,,,,,,,,,,,,,,,,
ec61bab6e46741969bceb80ee2246983,2019-05-26 12:06:22,50,50,50,50,intersex,25,1,80,58,more than 10 times,MSW,Path_ MS Wissenschaft-RadioTalk-ec61bab6e46741...,10.03.2020 01:11,RadioTalk,3070
1fbde03890d94bddaf7aa987fc732607,2019-05-26 16:02:45,100,100,100,96,Male,13,0,0,5,once,MSW,Path_ MS Wissenschaft-RadioTalk-1fbde03890d94b...,18.02.2020 16:24,RadioTalk,3070
1ad9eea672a94518b150925bac41fcbc,2019-05-26 17:23:38,17,22,0,86,Female,24,0,4,2,once,MSW,Path_ MS Wissenschaft-RadioTalk-1ad9eea672a945...,09.03.2020 21:31,RadioTalk,3070
461b2d80132f47958e50003c16084cf4,2019-05-26 17:27:34,30,70,80,80,Male,22,0,3,1,less than 10 times,MSW,Path_ MS Wissenschaft-TaxiDriver-461b2d80132f4...,10.03.2020 02:40,TaxiDriver,3070
4f300efb4a9941f7a96f57e8030b12e2,2019-05-26 17:39:25,100,100,50,50,Male,16,0,0,3,less than 10 times,MSW,Path_ MS Wissenschaft-RadioTalk-4f300efb4a9941...,09.03.2020 22:19,RadioTalk,3070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eda9175e9e1d4153ac2ca3038b2ab008,2019-12-23 09:23:08,65,30,33,20,Male,12,0,0,25,once,BMBF,Path_ MS Wissenschaft-RadioTalk-eda9175e9e1d41...,24.01.2020 13:28,RadioTalk,3070
b8c9e625f07444cc8d4a0fc245bab5dd,2019-12-23 09:28:25,100,50,50,100,Female,12,1,99,9,less than 10 times,BMBF,Path_ MS Wissenschaft-TaxiDriver-b8c9e625f0744...,24.01.2020 15:06,TaxiDriver,3070
2cd78ad9a37945de9f8cfd5f81c42082,2019-12-27 12:56:23,86,20,71,69,Male,57,0,39,0,never,BMBF,Path_ MS Wissenschaft-RadioTalk-2cd78ad9a37945...,24.01.2020 11:49,RadioTalk,3070


Update counters and show them

In [12]:
# last colliders counters
avas_c = wd_colls[wd_colls.condition == "AVAS"].index.size
radio_c = wd_colls[wd_colls.condition == "RadioTalk"].index.size
taxi_c = wd_colls[wd_colls.condition == "TaxiDriver"].index.size
wd_colls_c = wd_colls.index.size

# add and show them
counters = [avas_c, radio_c, taxi_c, wd_colls_c]
n_colls.loc["matched"] = counters
display(n_colls)

# last questionnaire counters
msw_c = wd_quest[wd_quest.expo == "MSW"].index.size
bmbf_c = wd_quest[wd_quest.expo == "BMBF"].index.size
wd_quest_c = wd_quest.index.size

# add and show them
counters = [msw_c, bmbf_c, wd_quest_c]  # arrange as list with total
n_quests.loc["matched"] = counters  # add current counters
display(n_quests)

# create participants counter
part_c_cols = ["avas", "radio", "taxi", "msw", "bmbf", "total"]  # df cols
n_participants = pd.DataFrame(columns=part_c_cols)  # create df
n_participants.index.name = "participants"  # set df name
# get counters
avas_c = wd_data[wd_data.condition == "AVAS"].index.size
radio_c = wd_data[wd_data.condition == "RadioTalk"].index.size
taxi_c = wd_data[wd_data.condition == "TaxiDriver"].index.size
msw_c = wd_data[wd_data.expo == "MSW"].index.size
bmbf_c = wd_data[wd_data.expo == "BMBF"].index.size
wd_data_c = wd_data.index.size

# add and show them
counters = [avas_c, radio_c, taxi_c, msw_c, bmbf_c, wd_quest_c]
n_participants.loc["matched"] = counters
n_participants

,avas,radio,taxi,total
colliders,,,,
raw,8896,8796,4428,22120
finished,8288,7928,4142,20358
matched,3284,3046,1583,7913


,msw,bmbf,total
questionnaries,,,
raw,8873,1209,10082
expo,8835,1153,9988
merged,8835,1153,9988
no duplicates,8731,1153,9884
no NaNs,8364,1153,9517
matched,6763,1150,7913


,avas,radio,taxi,msw,bmbf,total
participants,,,,,,
matched,3284,3046,1583,6763,1150,7913


__Store final merged dataframe as CSV (unify dataset)__

In [13]:
wd_data.to_csv("./data/participants.csv")